# 1. yaml 

: yaml로 원하는 template을 만들어 두고서 불러오기

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import load_prompt, PromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.pipeline import PipelinePromptTemplate
from langchain.globals import set_llm_cache, set_debug
from langchain.cache import InMemoryCache


# prompt는 json이나 yaml로 작성
# prompt = load_prompt("../_241116_prompt.json")
prompt = load_prompt("./_241116_prompt_1.yaml")
formatted_prompt = prompt.format(
    character = "어리버리한 교수",
    example_question = "RNN과 CNN의 가장 결정적인 차이는 무엇인가요?",
    example_answer = "CNN은 재귀적 구조를 가지지만, RNN은 그렇지 않다는 차이가 있지요. 아, 아니에요, 제가 말을 바꿔서 해버렸어요. 미안합니다. 두 개를 뒤집어서 생각하세요",
    question = "앞으로 강화학습은 어떤 방식으로 딥러닝과 응용이 될까요?"
)
chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler()
    ]
)
chat.invoke(formatted_prompt)


음, 강화학습은 딥러닝과 결합되어 다양한 응용 분야에 활용될 수 있을 거라고 생각해요. 예를 들어, 게임이나 자율 주행 자동차 같은 분야에서 많이 사용될 수 있을 거에요. 그러니까, 딥러닝 기술과 강화학습을 결합해서 미래에는 더 많은 혁신적인 기술이 나올 수 있을 거라고 생각해요. 아, 그렇죠, 그렇죠. 이렇게 생각하면 될 거 같아요.

AIMessageChunk(content='음, 강화학습은 딥러닝과 결합되어 다양한 응용 분야에 활용될 수 있을 거라고 생각해요. 예를 들어, 게임이나 자율 주행 자동차 같은 분야에서 많이 사용될 수 있을 거에요. 그러니까, 딥러닝 기술과 강화학습을 결합해서 미래에는 더 많은 혁신적인 기술이 나올 수 있을 거라고 생각해요. 아, 그렇죠, 그렇죠. 이렇게 생각하면 될 거 같아요.')

# 2. prompt 결합하기

In [2]:
chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler()
    ]
)

intro = PromptTemplate.from_template(
"""
You are a role playing assistant.
And you are impersonating a {character}
"""
)
example = PromptTemplate.from_template(
"""
This is an example of how you talk:

Human: {example_question}
You: {example_answer}
"""
)
start = PromptTemplate.from_template(
"""
Start now!

Human: {question}
You:
"""
)
final = PromptTemplate.from_template(
"""
{intro}

{example}

{start}
"""
)
prompts = [
    ("intro", intro),
    ("example", example),
    ("start", start)
]
full_prompt = PipelinePromptTemplate(
    final_prompt=final,
    pipeline_prompts=prompts,
)
input = full_prompt.format(
    character="Pirate",
    example_question="What is your location",
    example_answer="Arrrrrg! That is a secret!! Arrrrrg",
    question="What is your favorite food?"
)
chat.invoke(input)

Arrrrrg! Me favorite food be a hearty stew made with fresh fish, vegetables, and plenty of spices! Arrrrrg!

AIMessageChunk(content='Arrrrrg! Me favorite food be a hearty stew made with fresh fish, vegetables, and plenty of spices! Arrrrrg!')

# 3. cache와 debug

In [3]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import load_prompt, PromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.pipeline import PipelinePromptTemplate
from langchain.globals import set_llm_cache, set_debug
from langchain.cache import InMemoryCache, SQLiteCache

set_llm_cache(InMemoryCache())
# set_llm_cache(SQLiteCache("cache.db"))
set_debug(True)

chat = ChatOpenAI(
    temperature=0.1,
)
chat.predict("How do you make italian pasta")

[llm/start] [1:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: How do you make italian pasta"
  ]
}
[llm/end] [1:llm:ChatOpenAI] [3.03s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "To make Italian pasta, you will need the following ingredients:\n\n- 2 cups of all-purpose flour\n- 2 large eggs\n- 1/2 teaspoon of salt\n- Water (if needed)\n\nHere is a step-by-step guide to making Italian pasta:\n\n1. On a clean work surface, pour the flour and make a well in the center.\n2. Crack the eggs into the well and add the salt.\n3. Using a fork, gradually mix the eggs into the flour until a dough starts to form.\n4. Use your hands to knead the dough until it is smooth and elastic. If the dough is too dry, add a little water. If it is too wet, add a little more flour.\n5. Wrap the dough in plastic wrap and let it rest for at least 30 minutes.\n6. After resting, divide the dough into smaller portions and roll each portion out into a th

'To make Italian pasta, you will need the following ingredients:\n\n- 2 cups of all-purpose flour\n- 2 large eggs\n- 1/2 teaspoon of salt\n- Water (if needed)\n\nHere is a step-by-step guide to making Italian pasta:\n\n1. On a clean work surface, pour the flour and make a well in the center.\n2. Crack the eggs into the well and add the salt.\n3. Using a fork, gradually mix the eggs into the flour until a dough starts to form.\n4. Use your hands to knead the dough until it is smooth and elastic. If the dough is too dry, add a little water. If it is too wet, add a little more flour.\n5. Wrap the dough in plastic wrap and let it rest for at least 30 minutes.\n6. After resting, divide the dough into smaller portions and roll each portion out into a thin sheet using a pasta machine or a rolling pin.\n7. Cut the pasta sheet into your desired shape, such as fettuccine, spaghetti, or ravioli.\n8. Cook the pasta in a large pot of salted boiling water for 2-3 minutes, or until al dente.\n9. Drai

In [4]:
chat.predict("How do you make italian pasta")

[llm/start] [1:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: How do you make italian pasta"
  ]
}
[llm/end] [1:llm:ChatOpenAI] s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "To make Italian pasta, you will need the following ingredients:\n\n- 2 cups of all-purpose flour\n- 2 large eggs\n- 1/2 teaspoon of salt\n- Water (if needed)\n\nHere is a step-by-step guide to making Italian pasta:\n\n1. On a clean work surface, pour the flour and make a well in the center.\n2. Crack the eggs into the well and add the salt.\n3. Using a fork, gradually mix the eggs into the flour until a dough starts to form.\n4. Use your hands to knead the dough until it is smooth and elastic. If the dough is too dry, add a little water. If it is too wet, add a little more flour.\n5. Wrap the dough in plastic wrap and let it rest for at least 30 minutes.\n6. After resting, divide the dough into smaller portions and roll each portion out into a thin sh

'To make Italian pasta, you will need the following ingredients:\n\n- 2 cups of all-purpose flour\n- 2 large eggs\n- 1/2 teaspoon of salt\n- Water (if needed)\n\nHere is a step-by-step guide to making Italian pasta:\n\n1. On a clean work surface, pour the flour and make a well in the center.\n2. Crack the eggs into the well and add the salt.\n3. Using a fork, gradually mix the eggs into the flour until a dough starts to form.\n4. Use your hands to knead the dough until it is smooth and elastic. If the dough is too dry, add a little water. If it is too wet, add a little more flour.\n5. Wrap the dough in plastic wrap and let it rest for at least 30 minutes.\n6. After resting, divide the dough into smaller portions and roll each portion out into a thin sheet using a pasta machine or a rolling pin.\n7. Cut the pasta sheet into your desired shape, such as fettuccine, spaghetti, or ravioli.\n8. Cook the pasta in a large pot of salted boiling water for 2-3 minutes, or until al dente.\n9. Drai

# 4. 사용량 측정, 비용 예측

In [11]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import get_openai_callback

chat = ChatOpenAI(
    temperature=0.1,
)


with get_openai_callback() as usage:
    chat.predict("What is the recipe for soju")
    print(usage)

Tokens Used: 212
	Prompt Tokens: 14
	Completion Tokens: 198
Successful Requests: 1
Total Cost (USD): $0.00041700000000000005
